# Домашнее задание № 2. Мешок слов

In [397]:
import pandas as pd
from razdel import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report
import nltk
from pymorphy2 import MorphAnalyzer
from sklearn.naive_bayes import GaussianNB, BernoulliNB

stopwords = nltk.corpus.stopwords.words("russian")
pymorphy = MorphAnalyzer()

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [7]:
def razdel_tokenize_wrapper(text, *args, **kwargs):
    tokens = tokenize(text, *args, **kwargs)
    return [token.text for token in tokens]


data = pd.read_csv("labeled.csv")
X, y = data.comment, data.toxic
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

default_vectorizer = TfidfVectorizer()
razdel_vectorizer = TfidfVectorizer(tokenizer=razdel_tokenize_wrapper)

lr_default = LogisticRegression()
lr_razdel = LogisticRegression()

In [ ]:
default_train = default_vectorizer.fit_transform(X_train)
razdel_train = razdel_vectorizer.fit_transform(X_train)

lr_default.fit(default_train, y_train)
lr_razdel.fit(razdel_train, y_train)

default_preds = lr_default.predict(default_vectorizer.transform(X_test))
razdel_preds = lr_razdel.predict(razdel_vectorizer.transform(X_test))

default_f1 = f1_score(default_preds, y_test)
razdel_f1 = f1_score(razdel_preds, y_test)

In [37]:
print(f"default tokenizer f1: {default_f1}")
print(f"razdel tokenizer f1: {razdel_f1}")

default tokenizer f1: 0.6722173531989483
razdel tokenizer f1: 0.691947166595654


## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [ ]:
def preprocessing(text):
    tokens = razdel_tokenize_wrapper(text)
    lemmas = [
        lemma
        for word in tokens
        if (lemma := pymorphy.parse(word)[0].normal_form) not in stopwords
    ]

    return " ".join(lemmas)

In [118]:
data.toxic.value_counts()

toxic
0.0    9586
1.0    4826
Name: count, dtype: int64

In [145]:
toxic = data[data.toxic == 1]
non_toxic = data[data.toxic == 0][: len(toxic)]
balanced_data = pd.concat([toxic, non_toxic])

X, y = balanced_data.comment, balanced_data.toxic
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True
)

### 1. Random forest + TfidfVec

In [146]:
vec_tfidf = TfidfVectorizer(
    max_features=10000,
    max_df=0.8,
    min_df=3,
    ngram_range=(1, 2),
    preprocessor=preprocessing,
)

In [147]:
X_vec = vec_tfidf.fit_transform(X_train)

In [171]:
random_forest = RandomForestClassifier(n_estimators=120, max_features=0.2)
random_forest.fit(X_vec, y_train)
X_test_vec = vec_tfidf.transform(X_test)
forest_preds = random_forest.predict(X_test_vec)

In [172]:
print(classification_report(forest_preds, y_test))

              precision    recall  f1-score   support

         0.0       0.75      0.74      0.75      1450
         1.0       0.75      0.75      0.75      1446

    accuracy                           0.75      2896
   macro avg       0.75      0.75      0.75      2896
weighted avg       0.75      0.75      0.75      2896



### 2. LogReg + CountVec

In [175]:
vec_count = CountVectorizer(
    max_features=10000,
    max_df=0.7,
    min_df=10,
    ngram_range=(1, 1),
    preprocessor=preprocessing,
)

In [207]:
X_vec = vec_count.fit_transform(X_train)

In [359]:
lr = LogisticRegression(penalty="l1", max_iter=500, solver="liblinear")
lr.fit(X_vec, y_train)
X_test_vec = vec_count.transform(X_test)
lr_preds = lr.predict(X_test_vec)

In [182]:
print(classification_report(lr_preds, y_test))

              precision    recall  f1-score   support

         0.0       0.75      0.81      0.78      1343
         1.0       0.82      0.77      0.80      1553

    accuracy                           0.79      2896
   macro avg       0.79      0.79      0.79      2896
weighted avg       0.79      0.79      0.79      2896



## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [297]:
def print_top5_toxic(coefs):
    vocab = vec_count.get_feature_names_out()
    top5_indexes = np.argsort(-coefs)[:5]
    top_5_toxic_words = [vocab[i] for i in top5_indexes]
    print("\n".join(top_5_toxic_words))


def print_top5_toxic_tree(tree):
    train_predictions = tree.predict(X_vec)
    toxic_features = []
    non_toxic_features = []
    vocab = vec_count.get_feature_names_out()
    features_importance = tree.feature_importances_
    top_50_candidates = np.argsort(-features_importance)[:50].tolist()

    while len(toxic_features) < 5:
        current_feat = top_50_candidates.pop(0)
        toxic_cr = 0
        non_toxic_cr = 0
        for i in range(len(train_predictions)):
            feat_value = X_vec[i, current_feat]
            # 0.5 порог на всех разбиениях (я посмотрела на картинку дерева)
            if feat_value >= 0.5:
                if train_predictions[i] == 0:
                    non_toxic_cr += 1
                else:
                    toxic_cr += 1

        target_list = toxic_features if toxic_cr > non_toxic_cr else non_toxic_features
        target_list.append(current_feat)

    toxic_words_tree = [vocab[i] for i in toxic_features]
    print("\n".join(toxic_words_tree))

#### 1. LR

In [298]:
# lr from 2 section
print_top5_toxic(lr.coef_[0])

дебил
хохол
рашка
вон
дегенерат


#### 2. Decision Tree

In [354]:
tree = DecisionTreeClassifier(
    criterion="log_loss", min_samples_leaf=3, max_features="sqrt"
)
tree.fit(X_vec, y_train)
tree_preds = tree.predict(X_test_vec)
print(classification_report(tree_preds, y_test))

              precision    recall  f1-score   support

         0.0       0.75      0.71      0.73      1510
         1.0       0.70      0.74      0.72      1386

    accuracy                           0.73      2896
   macro avg       0.73      0.73      0.73      2896
weighted avg       0.73      0.73      0.73      2896



In [457]:
print_top5_toxic_tree(tree)

хохлов
дебил
хохол
тред
тупой


### 3. Random Forest

In [455]:
rf = RandomForestClassifier(n_estimators=130, max_depth=25)
rf.fit(X_vec, y_train)
rf = RandomForestClassifier(
    n_estimators=130, max_depth=25, criterion="log_loss", max_features="sqrt"
)
rf.fit(X_vec, y_train)
rf_preds = rf.predict(X_test_vec)
print(classification_report(rf_preds, y_test))

              precision    recall  f1-score   support

         0.0       0.67      0.76      0.71      1260
         1.0       0.80      0.71      0.75      1636

    accuracy                           0.73      2896
   macro avg       0.73      0.74      0.73      2896
weighted avg       0.74      0.73      0.73      2896



In [458]:
print_top5_toxic_tree(rf)

хохол
хохлов
тупой
нахуй
блядь


#### 4. NaiveBayes

In [430]:
nb = BernoulliNB(alpha=0.001, class_prior=[0.5, 0.5])
nb.fit(X_vec.toarray(), y_train)
nb_preds = nb.predict(X_test_vec.toarray())
print(classification_report(nb_preds, y_test))

              precision    recall  f1-score   support

         0.0       0.62      0.86      0.72      1035
         1.0       0.90      0.70      0.79      1861

    accuracy                           0.76      2896
   macro avg       0.76      0.78      0.75      2896
weighted avg       0.80      0.76      0.76      2896



In [421]:
print_top5_toxic(nb.feature_log_prob_[1, :])  # стоп-слова из нлтк не прокатили ((

это
всё
свой
весь
ещё
